In [3]:
# First, let's check the version of langchain
!pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [5]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.77
    Uninstalling langchain-core-0.3.77:
      Successfully uninstalled langchain-core-0.3.77


In [6]:
# Import the required dependencies
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

In [28]:
# Initialize the chat model with consistent settings
chat = ChatOpenAI(model_name = 'deepseek/deepseek-chat-v3.1:free',
                  temperature = 2,
                  max_tokens = 100,
                  api_key="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",
                  base_url="https://openrouter.ai/api/v1")

# 1. String Output Parser



In [29]:
# Import the String Output Parser
from langchain_core.output_parsers import StrOutputParser

In [30]:
# Create a message asking for an interesting fact
message_h = HumanMessage(content = "Can you give me an interesting fact I probably didn't know about?")

In [31]:
# Get the response from the chat model
response = chat.invoke([message_h])

# Show the raw response
print("Raw response:")
print(response)

Raw response:
content='Of course! Here\'s one that often surprises people, from the realm of linguistics:\n\n**The word "avocado" comes from the Nahuatl word *āhuacatl*, which literally means "testicle."**\n\nThe Aztecs named the fruit for its shape and the way it grows in pairs on the tree. When the Spanish encountered it, they adapted the word to the more pronounceable (for them) "aguacate," which eventually became "avocado" in English.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 18, 'total_tokens': 118, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3.1:free', 'system_fingerprint': None, 'id': 'gen-1759811913-ghJBlQGPvRVJUdR5NYo9', 'service_tier': None, 'finish_reason': 'length', 'logprobs': None} id='run--2a61cf3f-7e28-44a4-8758-5744ead378a7-0' usage_metadata={'input_tokens': 18, 'output_tokens': 100, 'total_tokens': 118, 'input_token_details': {}, 'outp

In [32]:
# Create and use the string output parser
str_output_parser = StrOutputParser()
response_parsed = str_output_parser.invoke(response)

print("Parsed response:")
print(response_parsed)

Parsed response:
Of course! Here's one that often surprises people, from the realm of linguistics:

**The word "avocado" comes from the Nahuatl word *āhuacatl*, which literally means "testicle."**

The Aztecs named the fruit for its shape and the way it grows in pairs on the tree. When the Spanish encountered it, they adapted the word to the more pronounceable (for them) "aguacate," which eventually became "avocado" in English.


# 2. Comma-Separated List Output Parser

The Comma-Separated List Output Parser helps in getting responses in the form of a list, where items are separated by commas. This is particularly useful when you want to get multiple items as a structured list from the model.

In [33]:
# Import the Comma-Separated List Output Parser
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [34]:
# Create the list parser and get its format instructions
list_output_parser = CommaSeparatedListOutputParser()
print("Format Instructions:")
print(list_output_parser.get_format_instructions())

Format Instructions:
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [35]:
# Create a message asking for dog names with format instructions
message_h = HumanMessage(content = f'''I've recently adopted a dog. Could you suggest some dog names?

{list_output_parser.get_format_instructions()}
''')

In [36]:
# Get response from the chat model
response = chat.invoke([message_h])
print("Raw response:")
print(response.content)

Raw response:
Bella, Luna, Charlie, Max, Milo, Bailey, Cooper, Rocky, Lucy, Daisy


In [37]:
# Parse the response into a list
response_parsed = list_output_parser.invoke(response)

print("\nParsed response (as a list):")
print(response_parsed)


Parsed response (as a list):
['Bella', 'Luna', 'Charlie', 'Max', 'Milo', 'Bailey', 'Cooper', 'Rocky', 'Lucy', 'Daisy']


# 3. Datetime Output Parser

The Datetime Output Parser is specialized in extracting and parsing datetime information from the model's responses. It ensures that dates and times are returned in a structured datetime format that can be easily used in Python.

In [38]:
# Import the Datetime Output Parser
from langchain.output_parsers import DatetimeOutputParser

In [39]:
# Create the datetime parser and show its format instructions
date_output_parser = DatetimeOutputParser()
print("DateTime Parser Format Instructions:")
print(date_output_parser.get_format_instructions())

DateTime Parser Format Instructions:
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z

Return ONLY this string, no other words!


In [40]:
# Create a message asking about a historical date
message_h = HumanMessage(content = f'''When was the Danish poet Piet Hein born?
{date_output_parser.get_format_instructions()}
''')

In [41]:
# Get response from the chat model
response = chat.invoke([message_h])
print("Raw response:")
print(response.content)

Raw response:
Piet Hein (the Danish poet and scientist) was born on December 16, 1905.

1905-12-16T00:00:00.000000Z


In [43]:
# Parse the response into a datetime object
# Extract only the datetime string from the response content
datetime_string = response.content.splitlines()[-1]
response_parsed = date_output_parser.invoke(datetime_string)

print("\nParsed response (as datetime):")
print(response_parsed)


Parsed response (as datetime):
1905-12-16 00:00:00
